In [138]:
import h2o
from bayes_opt import BayesianOptimization

In [147]:
h2o.init()
h2o.remove_all()
h2o.no_progress()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,22 hours 40 mins
H2O cluster timezone:,America/Santiago
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.3
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_maravenag_4bxahk
H2O cluster total nodes:,1
H2O cluster free memory:,3.495 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [148]:
data = h2o.upload_file("data/winequality-red.csv",destination_frame="train_frame")

In [149]:
data.head(10)

fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8,5
7.8,0.76,0.04,2.3,0.092,15,54,0.997,3.26,0.65,9.8,5
11.2,0.28,0.56,1.9,0.075,17,60,0.998,3.16,0.58,9.8,6
7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
7.4,0.66,0,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5
7.9,0.6,0.06,1.6,0.069,15,59,0.9964,3.3,0.46,9.4,5
7.3,0.65,0,1.2,0.065,15,21,0.9946,3.39,0.47,10,7
7.8,0.58,0.02,2,0.073,9,18,0.9968,3.36,0.57,9.5,7
7.5,0.5,0.36,6.1,0.071,17,102,0.9978,3.35,0.8,10.5,5


In [150]:
train_cols = [x for x in data.col_names if x not in ['quality'] ] 

In [151]:
train, test = data.split_frame(ratios=[0.7], destination_frames=['train', 'test'])

In [152]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [153]:
def train_model(max_depth, 
                ntrees,
                min_rows, 
                learn_rate, 
                sample_rate, 
                col_sample_rate):
    params = {
        'max_depth': int(max_depth),
        'ntrees': int(ntrees),
        'min_rows': int(min_rows),
        'learn_rate':learn_rate,
        'sample_rate':sample_rate,
        'col_sample_rate':col_sample_rate
    }
    model = H2OGradientBoostingEstimator(nfolds=5,**params)
    model.train(x=train_cols, y='quality', training_frame=train)
    h2o_data = h2o.ls()
    h2o_data = list(h2o_data['key'])
    h2o_data = list(filter(lambda x: x not in ["train_frame", "train", "test"], h2o_data))
    metric = -model.rmse()
    list(map(lambda x: h2o.remove(x), h2o_data))
    return metric

In [154]:
bounds = {
    'max_depth':(5,10),
    'ntrees': (100,500),
    'min_rows':(10,30),
    'learn_rate':(0.001, 0.01),
    'sample_rate':(0.5,0.8),
    'col_sample_rate':(0.5,0.8)
}

In [155]:
optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
)

In [156]:
optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   | col_sa... | learn_... | max_depth | min_rows  |  ntrees   | sample... |
-------------------------------------------------------------------------------------------------
|  1        | -0.6069   |  0.6251   |  0.007483 |  5.001    |  16.05    |  158.7    |  0.5277   |
|  2        | -0.6005   |  0.5559   |  0.00411  |  6.984    |  20.78    |  267.7    |  0.7056   |
|  3        | -0.5531   |  0.5613   |  0.008903 |  5.137    |  23.41    |  266.9    |  0.6676   |
|  4        | -0.6697   |  0.5421   |  0.002783 |  9.004    |  29.37    |  225.4    |  0.7077   |
|  5        | -0.5527   |  0.7629   |  0.009051 |  5.425    |  10.78    |  167.9    |  0.7634   |
|  6        | -0.5509   |  0.5295   |  0.00479  |  9.789    |  20.66    |  376.8    |  0.5947   |
|  7        | -0.5153   |  0.706    |  0.008512 |  5.091    |  25.0     |  495.5    |  0.7244   |
|  8        | -0.5175   |  0.5841   |  0.008104 |  5.516    |  18.96    |  463.4    |  0.5881   |
|  9        | -0.711

In [157]:
optimizer.max

{'target': -0.2920199343134594,
 'params': {'col_sample_rate': 0.8,
  'learn_rate': 0.009999999708992287,
  'max_depth': 10.0,
  'min_rows': 10.0,
  'ntrees': 500.0,
  'sample_rate': 0.8}}